In [1]:
import sys
sys.path.insert(0, sys.path[0].removesuffix('/src/jupyter_nb'))

import pandas as pd
from pycaret.classification import ClassificationExperiment
import src.utils as utils
import src.calcEMA as calc_utils
import src.myenv as myenv
from datetime import datetime

from itertools import combinations
import plotly.express as px

<built-in method __dir__ of sqlite3.Connection object at 0x7fc759656240>


In [2]:
# Variables
# ETCUSDT BTCUSDT
# symbol = 'ETHUSDT'
symbol = 'BTCUSDT'
interval = '1h'
# lightgbm  xgboost lr et rf
estimator = 'knn'
_compare_models = False

start_train_date = '2010-01-01'  # train < and test >=
start_test_date = '2023-01-01'  # train < and test >=

stop_loss = 1.0
# regression_times = 0  # 24 * 30 * 2  # horas
times_regression_PnL = 120
normalize = True
use_gpu = False
tune_model = False
apply_combination_features = False

### Metadata

<code>
Field Name - Description</br>
open_time - Kline Open time in unix time format</br>
open - Open Price</br>
high - High Price</br>
low	- Low Price</br>
close	- Close Price</br>
volume - Volume</br>
close_time - Kline Close time in unix time format</br>
quote_volume - Quote Asset Volume</br>
count	- Number of Trades</br>
taker_buy_volume - Taker buy base asset volume during this period</br>
taker_buy_quote_volume - Taker buy quote asset volume during this period</br>
ignore - Ignore</br>
</code>

In [3]:
cols = myenv.all_klines_cols.copy()
cols.remove('ignore')
data = utils.get_data(symbol=symbol, save_database=False, interval=interval, tail=-1, columns=cols, parse_dates=False)
data = utils.parse_type_fields(data, parse_dates=True)
data = utils.adjust_index(data)
data.info()
data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 53905 entries, 2017-08-17 04:00:00 to 2023-10-16 11:00:00
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     53905 non-null  datetime64[ns]
 1   open                          53905 non-null  float32       
 2   high                          53905 non-null  float32       
 3   low                           53905 non-null  float32       
 4   close                         53905 non-null  float32       
 5   volume                        53905 non-null  float32       
 6   close_time                    53905 non-null  int64         
 7   quote_asset_volume            53905 non-null  float32       
 8   number_of_trades              53905 non-null  int16         
 9   taker_buy_base_asset_volume   53905 non-null  float32       
 10  taker_buy_quote_asset_volume  53905 non-null  float32      

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol
ix_open_time,,,,,,,,,,,,
2017-08-17 04:00:00,2017-08-17 04:00:00,4261.479980,4313.620117,4261.319824,4308.830078,47.181007,1502945999999,2.023661e+05,171,35.160503,1.509525e+05,BTCUSDT
2017-08-17 05:00:00,2017-08-17 05:00:00,4308.830078,4328.689941,4291.370117,4315.319824,23.234917,1502949599999,1.003048e+05,102,21.448071,9.260828e+04,BTCUSDT
2017-08-17 06:00:00,2017-08-17 06:00:00,4330.290039,4345.450195,4309.370117,4324.350098,7.229691,1502953199999,3.128231e+04,36,4.802861,2.079532e+04,BTCUSDT
2017-08-17 07:00:00,2017-08-17 07:00:00,4316.620117,4349.990234,4287.410156,4349.990234,4.443249,1502956799999,1.924106e+04,25,2.602292,1.129135e+04,BTCUSDT
2017-08-17 08:00:00,2017-08-17 08:00:00,4333.319824,4377.850098,4333.319824,4360.689941,0.972807,1502960399999,4.239503e+03,28,0.814655,3.552747e+03,BTCUSDT
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-16 07:00:00,2023-10-16 07:00:00,27812.599609,27920.000000,27802.939453,27849.060547,2012.976074,1697443199999,5.609566e+07,-12261,1137.027344,3.168454e+07,BTCUSDT
2023-10-16 08:00:00,2023-10-16 08:00:00,27849.050781,27902.990234,27805.289062,27809.839844,1725.431763,1697446799999,4.806433e+07,-21434,795.355225,2.215686e+07,BTCUSDT
2023-10-16 09:00:00,2023-10-16 09:00:00,27809.830078,27810.939453,27641.720703,27768.070312,2028.940063,1697450399999,5.623989e+07,2211,980.163269,2.716522e+07,BTCUSDT


In [4]:
data = calc_utils.calc_RSI(data)
data.info()
data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 53905 entries, 2017-08-17 04:00:00 to 2023-10-16 11:00:00
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     53905 non-null  datetime64[ns]
 1   open                          53905 non-null  float32       
 2   high                          53905 non-null  float32       
 3   low                           53905 non-null  float32       
 4   close                         53905 non-null  float32       
 5   volume                        53905 non-null  float32       
 6   close_time                    53905 non-null  int64         
 7   quote_asset_volume            53905 non-null  float32       
 8   number_of_trades              53905 non-null  int16         
 9   taker_buy_base_asset_volume   53905 non-null  float32       
 10  taker_buy_quote_asset_volume  53905 non-null  float32      

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,rsi
ix_open_time,,,,,,,,,,,,,
2017-08-17 04:00:00,2017-08-17 04:00:00,4261.479980,4313.620117,4261.319824,4308.830078,47.181007,1502945999999,2.023661e+05,171,35.160503,1.509525e+05,BTCUSDT,NaN
2017-08-17 05:00:00,2017-08-17 05:00:00,4308.830078,4328.689941,4291.370117,4315.319824,23.234917,1502949599999,1.003048e+05,102,21.448071,9.260828e+04,BTCUSDT,NaN
2017-08-17 06:00:00,2017-08-17 06:00:00,4330.290039,4345.450195,4309.370117,4324.350098,7.229691,1502953199999,3.128231e+04,36,4.802861,2.079532e+04,BTCUSDT,NaN
2017-08-17 07:00:00,2017-08-17 07:00:00,4316.620117,4349.990234,4287.410156,4349.990234,4.443249,1502956799999,1.924106e+04,25,2.602292,1.129135e+04,BTCUSDT,NaN
2017-08-17 08:00:00,2017-08-17 08:00:00,4333.319824,4377.850098,4333.319824,4360.689941,0.972807,1502960399999,4.239503e+03,28,0.814655,3.552747e+03,BTCUSDT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-16 07:00:00,2023-10-16 07:00:00,27812.599609,27920.000000,27802.939453,27849.060547,2012.976074,1697443199999,5.609566e+07,-12261,1137.027344,3.168454e+07,BTCUSDT,81.683762
2023-10-16 08:00:00,2023-10-16 08:00:00,27849.050781,27902.990234,27805.289062,27809.839844,1725.431763,1697446799999,4.806433e+07,-21434,795.355225,2.215686e+07,BTCUSDT,78.638657
2023-10-16 09:00:00,2023-10-16 09:00:00,27809.830078,27810.939453,27641.720703,27768.070312,2028.940063,1697450399999,5.623989e+07,2211,980.163269,2.716522e+07,BTCUSDT,75.414246


In [5]:
data = calc_utils.calc_ema_periods(data, periods_of_time=[times_regression_PnL, 200])
data.info()
data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 53905 entries, 2017-08-17 04:00:00 to 2023-10-16 11:00:00
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     53905 non-null  datetime64[ns]
 1   open                          53905 non-null  float32       
 2   high                          53905 non-null  float32       
 3   low                           53905 non-null  float32       
 4   close                         53905 non-null  float32       
 5   volume                        53905 non-null  float32       
 6   close_time                    53905 non-null  int64         
 7   quote_asset_volume            53905 non-null  float32       
 8   number_of_trades              53905 non-null  int16         
 9   taker_buy_base_asset_volume   53905 non-null  float32       
 10  taker_buy_quote_asset_volume  53905 non-null  float32      

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,rsi,ema_120p,ema_120p_diff,ema_200p,ema_200p_diff
ix_open_time,,,,,,,,,,,,,,,,,
2017-08-17 04:00:00,2017-08-17 04:00:00,4261.479980,4313.620117,4261.319824,4308.830078,47.181007,1502945999999,2.023661e+05,171,35.160503,1.509525e+05,BTCUSDT,NaN,NaN,NaN,NaN,NaN
2017-08-17 05:00:00,2017-08-17 05:00:00,4308.830078,4328.689941,4291.370117,4315.319824,23.234917,1502949599999,1.003048e+05,102,21.448071,9.260828e+04,BTCUSDT,NaN,NaN,NaN,NaN,NaN
2017-08-17 06:00:00,2017-08-17 06:00:00,4330.290039,4345.450195,4309.370117,4324.350098,7.229691,1502953199999,3.128231e+04,36,4.802861,2.079532e+04,BTCUSDT,NaN,NaN,NaN,NaN,NaN
2017-08-17 07:00:00,2017-08-17 07:00:00,4316.620117,4349.990234,4287.410156,4349.990234,4.443249,1502956799999,1.924106e+04,25,2.602292,1.129135e+04,BTCUSDT,NaN,NaN,NaN,NaN,NaN
2017-08-17 08:00:00,2017-08-17 08:00:00,4333.319824,4377.850098,4333.319824,4360.689941,0.972807,1502960399999,4.239503e+03,28,0.814655,3.552747e+03,BTCUSDT,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-16 07:00:00,2023-10-16 07:00:00,27812.599609,27920.000000,27802.939453,27849.060547,2012.976074,1697443199999,5.609566e+07,-12261,1137.027344,3.168454e+07,BTCUSDT,81.683762,27055.065434,2.934737,27126.643747,2.663126
2023-10-16 08:00:00,2023-10-16 08:00:00,27849.050781,27902.990234,27805.289062,27809.839844,1725.431763,1697446799999,4.806433e+07,-21434,795.355225,2.215686e+07,BTCUSDT,78.638657,27067.541044,2.742395,27133.441718,2.492858
2023-10-16 09:00:00,2023-10-16 09:00:00,27809.830078,27810.939453,27641.720703,27768.070312,2028.940063,1697450399999,5.623989e+07,2211,980.163269,2.716522e+07,BTCUSDT,75.414246,27079.120040,2.544212,27139.756430,2.315105


In [6]:
data = utils.regression_PnL(
    data=data,
    label=myenv.label,
    diff_percent=float(stop_loss),
    max_regression_profit_and_loss=int(times_regression_PnL),
    drop_na=True,
    drop_calc_cols=True,
    strategy=None)
data.info()
data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 53586 entries, 2017-08-25 11:00:00 to 2023-10-11 11:00:00
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     53586 non-null  datetime64[ns]
 1   open                          53586 non-null  float32       
 2   high                          53586 non-null  float32       
 3   low                           53586 non-null  float32       
 4   close                         53586 non-null  float32       
 5   volume                        53586 non-null  float32       
 6   close_time                    53586 non-null  int64         
 7   quote_asset_volume            53586 non-null  float32       
 8   number_of_trades              53586 non-null  int16         
 9   taker_buy_base_asset_volume   53586 non-null  float32       
 10  taker_buy_quote_asset_volume  53586 non-null  float32      

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,rsi,ema_120p,ema_120p_diff,ema_200p,ema_200p_diff,status
ix_open_time,,,,,,,,,,,,,,,,,,
2017-08-25 11:00:00,2017-08-25 11:00:00,4331.669922,4361.279785,4313.000000,4361.270020,25.795115,1503662399999,1.118111e+05,216,8.989599,3.906715e+04,BTCUSDT,62.460861,4148.287323,5.134232,4132.685234,5.531144,CAI_1.0
2017-08-25 12:00:00,2017-08-25 12:00:00,4361.270020,4361.270020,4261.129883,4309.689941,110.998611,1503665999999,4.792662e+05,1891,7.918988,3.425532e+04,BTCUSDT,54.577419,4151.052128,3.821629,4134.718968,4.231750,SOBE_1.0
2017-08-25 13:00:00,2017-08-25 13:00:00,4311.689941,4345.000000,4297.660156,4312.000000,20.801018,1503669599999,8.978516e+04,188,4.482264,1.937958e+04,BTCUSDT,54.852245,4153.807489,3.808374,4136.752745,4.236348,SOBE_1.0
2017-08-25 14:00:00,2017-08-25 14:00:00,4303.819824,4340.000000,4291.520020,4305.000000,10.610266,1503673199999,4.580427e+04,139,6.107378,2.639153e+04,BTCUSDT,53.790161,4156.394313,3.575351,4138.679954,4.018674,SOBE_1.0
2017-08-25 15:00:00,2017-08-25 15:00:00,4321.500000,4422.290039,4321.500000,4394.359863,21.796762,1503676799999,9.534934e+04,189,5.421801,2.366765e+04,BTCUSDT,63.504879,4160.463417,5.621884,4141.604271,6.102843,SOBE_1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-11 07:00:00,2023-10-11 07:00:00,27068.960938,27099.529297,27018.560547,27021.949219,834.261047,1697011199999,2.257534e+07,31910,390.447449,1.056537e+07,BTCUSDT,29.502731,27587.912616,-2.051490,27554.285441,-1.931954,CAI_1.0
2023-10-11 08:00:00,2023-10-11 08:00:00,27021.949219,27117.310547,27017.000000,27108.429688,1013.988281,1697014799999,2.744929e+07,-29038,552.181641,1.494734e+07,BTCUSDT,36.503422,27579.987278,-1.709782,27549.849066,-1.602257,CAI_1.0
2023-10-11 09:00:00,2023-10-11 09:00:00,27108.439453,27288.980469,27090.009766,27285.439453,1686.210327,1697018399999,4.585223e+07,-4361,844.269531,2.295713e+07,BTCUSDT,47.906372,27575.118719,-1.050510,27547.218124,-0.950291,CAI_1.0


In [7]:
perc_data_label = data[[myenv.label, 'open_time']].groupby(myenv.label).count()
perc_data_label['perc'] = perc_data_label['open_time'] / len(data)
perc_data_label

,open_time,perc
status,,
CAI_1.0,26112,0.487291
ESTAVEL,92,0.001717
SOBE_1.0,27382,0.510992


In [8]:
train_data = data[(data['open_time'] >= start_train_date) & (data['open_time'] < start_test_date)]
train_data = train_data.sort_values(myenv.date_features)
train_data

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,rsi,ema_120p,ema_120p_diff,ema_200p,ema_200p_diff,status
ix_open_time,,,,,,,,,,,,,,,,,,
2017-08-25 11:00:00,2017-08-25 11:00:00,4331.669922,4361.279785,4313.000000,4361.270020,25.795115,1503662399999,1.118111e+05,216,8.989599,3.906715e+04,BTCUSDT,62.460861,4148.287323,5.134232,4132.685234,5.531144,CAI_1.0
2017-08-25 12:00:00,2017-08-25 12:00:00,4361.270020,4361.270020,4261.129883,4309.689941,110.998611,1503665999999,4.792662e+05,1891,7.918988,3.425532e+04,BTCUSDT,54.577419,4151.052128,3.821629,4134.718968,4.231750,SOBE_1.0
2017-08-25 13:00:00,2017-08-25 13:00:00,4311.689941,4345.000000,4297.660156,4312.000000,20.801018,1503669599999,8.978516e+04,188,4.482264,1.937958e+04,BTCUSDT,54.852245,4153.807489,3.808374,4136.752745,4.236348,SOBE_1.0
2017-08-25 14:00:00,2017-08-25 14:00:00,4303.819824,4340.000000,4291.520020,4305.000000,10.610266,1503673199999,4.580427e+04,139,6.107378,2.639153e+04,BTCUSDT,53.790161,4156.394313,3.575351,4138.679954,4.018674,SOBE_1.0
2017-08-25 15:00:00,2017-08-25 15:00:00,4321.500000,4422.290039,4321.500000,4394.359863,21.796762,1503676799999,9.534934e+04,189,5.421801,2.366765e+04,BTCUSDT,63.504879,4160.463417,5.621884,4141.604271,6.102843,SOBE_1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31 19:00:00,2022-12-31 19:00:00,16577.779297,16590.060547,16565.099609,16570.140625,4044.433594,1672516799999,6.704605e+07,-14485,2067.921631,3.428087e+07,BTCUSDT,50.101242,16628.054712,-0.348291,16683.831144,-0.681441,SOBE_1.0
2022-12-31 20:00:00,2022-12-31 20:00:00,16570.140625,16574.970703,16564.089844,16568.599609,2622.143555,1672520399999,4.344849e+07,25169,1311.381104,2.172951e+07,BTCUSDT,49.712753,16627.071983,-0.351670,16682.684562,-0.683852,SOBE_1.0
2022-12-31 21:00:00,2022-12-31 21:00:00,16568.189453,16571.640625,16544.119141,16548.279297,3618.773926,1672523999999,5.992803e+07,-26007,1746.413330,2.892190e+07,BTCUSDT,44.781689,16625.769624,-0.466086,16681.347196,-0.797705,SOBE_1.0


In [9]:
test_data = data[data['open_time'] >= start_test_date]
test_data = test_data.sort_values(myenv.date_features)
test_data

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,rsi,ema_120p,ema_120p_diff,ema_200p,ema_200p_diff,status
ix_open_time,,,,,,,,,,,,,,,,,,
2023-01-01 00:00:00,2023-01-01 00:00:00,16541.769531,16545.699219,16508.390625,16529.669922,4364.835938,1672534799999,72146296.0,18782,2179.947754,36032352.0,BTCUSDT,42.406311,16621.147989,-0.550372,16676.903422,-0.882859,SOBE_1.0
2023-01-01 01:00:00,2023-01-01 01:00:00,16529.589844,16556.800781,16525.779297,16551.470703,3590.066650,1672538399999,59376756.0,-4516,1730.249023,28617416.0,BTCUSDT,47.948830,16619.996298,-0.412308,16675.655335,-0.744706,SOBE_1.0
2023-01-01 02:00:00,2023-01-01 02:00:00,16551.470703,16559.769531,16538.140625,16548.189453,3318.840332,1672541999999,54919452.0,-15674,1611.123047,26660868.0,BTCUSDT,47.212383,16618.809408,-0.424940,16674.387018,-0.756835,SOBE_1.0
2023-01-01 03:00:00,2023-01-01 03:00:00,16548.189453,16548.189453,16518.210938,16533.039062,4242.080566,1672545599999,70122544.0,6652,2096.092773,34649044.0,BTCUSDT,43.862534,16617.391716,-0.507617,16672.980570,-0.839331,SOBE_1.0
2023-01-01 04:00:00,2023-01-01 04:00:00,16533.039062,16535.970703,16511.919922,16521.849609,4285.009277,1672549199999,70802648.0,-1537,2188.401855,36159820.0,BTCUSDT,41.519444,16615.812508,-0.565503,16671.476780,-0.897504,SOBE_1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-11 07:00:00,2023-10-11 07:00:00,27068.960938,27099.529297,27018.560547,27021.949219,834.261047,1697011199999,22575344.0,31910,390.447449,10565367.0,BTCUSDT,29.502731,27587.912616,-2.051490,27554.285441,-1.931954,CAI_1.0
2023-10-11 08:00:00,2023-10-11 08:00:00,27021.949219,27117.310547,27017.000000,27108.429688,1013.988281,1697014799999,27449286.0,-29038,552.181641,14947340.0,BTCUSDT,36.503422,27579.987278,-1.709782,27549.849066,-1.602257,CAI_1.0
2023-10-11 09:00:00,2023-10-11 09:00:00,27108.439453,27288.980469,27090.009766,27285.439453,1686.210327,1697018399999,45852232.0,-4361,844.269531,22957128.0,BTCUSDT,47.906372,27575.118719,-1.050510,27547.218124,-0.950291,CAI_1.0


In [10]:
# BTCUSDT 1h best params: close,volume,quote_asset_volume,number_of_trades,rsi
# numeric_features = 'volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,rsi,ema_24p,ema_200p'.split(',')
#text_numeric_features = 'close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,rsi,ema_24p,ema_200p'
text_numeric_features = 'number_of_trades,rsi,ema_120p,ema_200p'
numeric_features = text_numeric_features.split(',')
print(f'Numeric Features: {numeric_features} - size: {len(numeric_features)}\n')

combination_numeric_features = []
if apply_combination_features:
	for size in range(1, len(numeric_features) + 1): 
		comb = map(list, combinations(numeric_features, size))	
		for c in comb:
			res = ''
			for j in c:
				res += f'{j},'
			combination_numeric_features.append(res[0:len(res) - 1])
else:
	combination_numeric_features = [text_numeric_features]

print(f'Combination Numeric Features: \n{combination_numeric_features}')

Numeric Features: ['number_of_trades', 'rsi', 'ema_120p', 'ema_200p'] - size: 4

Combination Numeric Features: 
['number_of_trades,rsi,ema_120p,ema_200p']


In [11]:
from datetime import datetime
# fix_imbalance_method: condensednearestneighbour, editednearestneighborus, repeatededitednearestneighbours, allknn, instancehardnessthreshold, nearmiss, neighbourhoodcleaningrule, onesidedselection, randomundersampler, tomeklinks, randomoversampler, smote, smotenc, smoten, adasyn, borderlinesmote, kmeanssmote, svmsmote, smoteenn, smotetomek.
# 'condensednearestneighbour,editednearestneighborus,repeatededitednearestneighbours,allknn,instancehardnessthreshold,nearmiss,neighbourhoodcleaningrule,onesidedselection,randomundersampler,tomeklinks,randomoversampler,smote,smotenc,smoten,adasyn,borderlinesmote,kmeanssmote,svmsmote,smoteenn,smotetomek'.split(',')
# 'smotenc,smoten,adasyn,borderlinesmote,kmeanssmote,svmsmote,smoteenn,smotetomek'.split(',')
# imbalance_list = 'editednearestneighborus,repeatededitednearestneighbours,allknn,instancehardnessthreshold,nearmiss,neighbourhoodcleaningrule,onesidedselection,randomundersampler,tomeklinks,randomoversampler,smote,smotenc,smoten,adasyn,borderlinesmote,kmeanssmote,svmsmote,smoteenn,smotetomek'.split(',')
imbalance_list = 'instancehardnessthreshold,smoteenn,repeatededitednearestneighbours,allknn'.split(',')
simulation_results_filename = f'./resultado_simulacao_{symbol}_{interval}.csv'

df_resultado_simulacao = pd.read_csv(simulation_results_filename, sep=';')
for aux_numeric_features in combination_numeric_features:
  experiement = ClassificationExperiment()
  features = []
  features += ['open_time', myenv.label]
  features += aux_numeric_features.split(',')
  print(f'features: {features}')
  for fix_imbalance_method in imbalance_list:
    chave = (df_resultado_simulacao['symbol'] == symbol) & \
        (df_resultado_simulacao['interval'] == interval) & \
        (df_resultado_simulacao['estimator'] == estimator) & \
        (df_resultado_simulacao['fix_imbalance_method'] == fix_imbalance_method) & \
        (df_resultado_simulacao['features'] == str(features))
    
    if chave.sum() > 0:
      print(f'fix_imbalance_method: {fix_imbalance_method} already exists')
      continue

    try:
      print(f'fix_imbalance_method: {fix_imbalance_method}')
      setup = experiement.setup(
          data=train_data[features].copy(),
          train_size=myenv.train_size,
          target=myenv.label,
          numeric_features=aux_numeric_features.split(','),
          date_features=['open_time'],
          create_date_columns=["hour", "day", "month"],
          data_split_shuffle=False,
          data_split_stratify=False,
          fix_imbalance=True,
          fix_imbalance_method=fix_imbalance_method,
          remove_outliers=True,
          fold_strategy='timeseries',
          fold=3,
          session_id=123,
          normalize=normalize,
          use_gpu=use_gpu,
          verbose=True,
          n_jobs=20,
          log_experiment=False,
      )

      if _compare_models:
        best = setup.compare_models()
        estimator = setup.pull().index[0]
        print('Estimator: ' + estimator)
      else:
        best = setup.create_model(estimator)

      if tune_model:
        best = setup.tune_model(best)

      # predict on test set
      # holdout_pred = setup.predict_model(best)
      # print('Holdout Score:', holdout_pred['prediction_score'].mean())
      # print('Holdout Score Group:\n', holdout_pred[[myenv.label, 'prediction_score']].groupby(myenv.label).mean())

      predict = setup.predict_model(best, data=test_data.drop(columns=[myenv.label]))
      predict[myenv.label] = test_data[myenv.label]
      predict['_score'] = predict['prediction_label'] == predict[myenv.label]
      # print('Predict Score Mean:', predict['_score'].mean())
      # print('Predict Score Mean Group:\n', predict[[myenv.label, '_score']].groupby(myenv.label).mean())

      final_model = setup.finalize_model(best)

      ajusted_test_data = test_data.drop(myenv.label, axis=1)
      df_final_predict, res_score = utils.validate_score_test_data(
          setup,
          final_model,
          myenv.label,
          test_data,
          ajusted_test_data)

      # df_final_predict.info()
      # print('Final Score Mean:', res_score.mean().values[0])
      # print('Final Score Group:\n', res_score)

      start_test_date = df_final_predict['open_time'].min()
      end_test_date = df_final_predict['open_time'].max()

      # print('Simule Trading:')
      # print(f'Min Data: {start_test_date}')
      # print(f'Max Data: {end_test_date}')
      saldo_inicial = 100.0
      saldo_final = utils.simule_trading_crypto2(df_final_predict, start_test_date, end_test_date, saldo_inicial, stop_loss)
      print(f'>>>> Saldo Final: {saldo_final} - features: {features}\n\n')

      result_simulado = {}
      result_simulado['date'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
      result_simulado['symbol'] = symbol
      result_simulado['interval'] = interval
      result_simulado['estimator'] = estimator
      result_simulado['fix_imbalance_method'] = fix_imbalance_method
      result_simulado['stop_loss'] = stop_loss
      result_simulado['times_regression_profit_and_loss'] = times_regression_PnL
      result_simulado['features'] = features
      result_simulado['final_value'] = saldo_final

      if res_score is not None:
        result_simulado['score'] = ''
        for i in range(0, len(res_score.index.values)):
          result_simulado['score'] += f'[{res_score.index.values[i]}={res_score["_score"].values[i]:.2f}]'

      df_resultado_simulacao = pd.concat([df_resultado_simulacao, pd.DataFrame([result_simulado])], ignore_index=True)
      df_resultado_simulacao.sort_values('final_value', inplace=True)

      df_resultado_simulacao.to_csv(simulation_results_filename, sep=';', index=False)
    except Exception as e:
      print(e)
      continue

features: ['open_time', 'status', 'number_of_trades', 'rsi', 'ema_120p', 'ema_200p']
fix_imbalance_method: instancehardnessthreshold


,Description,Value
0,Session id,123
1,Target,status
2,Target type,Multiclass
3,Target mapping,"CAI_1.0: 0, ESTAVEL: 1, SOBE_1.0: 2"
4,Original data shape,"(46783, 6)"
5,Transformed data shape,"(14219, 8)"
6,Transformed train set shape,"(184, 8)"
7,Transformed test set shape,"(14035, 8)"
8,Numeric features,4
9,Date features,1


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,08:04:21
Status,. . . . . . . . . . . . . . . . . .,Fitting 3 Folds
Estimator,. . . . . . . . . . . . . . . . . .,K Neighbors Classifier


y contains previously unseen labels: 'ESTAVEL'
fix_imbalance_method: smoteenn


,Description,Value
0,Session id,123
1,Target,status
2,Target type,Multiclass
3,Target mapping,"CAI_1.0: 0, ESTAVEL: 1, SOBE_1.0: 2"
4,Original data shape,"(46783, 6)"
5,Transformed data shape,"(37519, 8)"
6,Transformed train set shape,"(23484, 8)"
7,Transformed test set shape,"(14035, 8)"
8,Numeric features,4
9,Date features,1


...

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,08:04:30
Status,. . . . . . . . . . . . . . . . . .,Fitting 3 Folds
Estimator,. . . . . . . . . . . . . . . . . .,K Neighbors Classifier


y contains previously unseen labels: 'ESTAVEL'
fix_imbalance_method: allknn


,Description,Value
0,Session id,123
1,Target,status
2,Target type,Multiclass
3,Target mapping,"CAI_1.0: 0, ESTAVEL: 1, SOBE_1.0: 2"
4,Original data shape,"(46783, 6)"
5,Transformed data shape,"(23908, 8)"
6,Transformed train set shape,"(9873, 8)"
7,Transformed test set shape,"(14035, 8)"
8,Numeric features,4
9,Date features,1


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,08:04:32
Status,. . . . . . . . . . . . . . . . . .,Fitting 3 Folds
Estimator,. . . . . . . . . . . . . . . . . .,K Neighbors Classifier


y contains previously unseen labels: 'ESTAVEL'
